<a href="https://colab.research.google.com/github/tsjaishankar1990/Advanced_Data_Science___my_learning/blob/main/gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/ce/cde/prod/sw/bin/env-switch python3
###############################################################################
# filename: spfMosIdsCheck_introduced_config_and_html

###############################################################################
# Description: Compare pre-layout vs post-layout transistor Ids to ensure
#              simulation health in DRAM design environments (DRAMCADDEV-2134).
# Programmer: bbcannon
# updated: jthommandram
# pending: verify algorithm, need to check the naming of parameters in configfile like: gatewidth / multiplier, etc., test html, etc. verifying original rtest  run for backward compatibility, and making it work in cetest dashboard!
###############################################################################

import argparse
import logging
import os
import shutil
import stat
import sys
import yaml
import re
import traceback  # Add this import for detailed error logging

import Model
from Test.TmpDesign import TmpDesign
from Model.Device.SPF.MosfetIdsCheck import MosfetIdsCheck
from Model.Device.Process import Process
from MUOA.DesignDatabase import DesignDatabase
from DTK2.DTK import DTK
from Utils.DateTime import DateTime
from Model.Sim.FinesimMosfetIdsSim import FinesimMosfetIdsSim, IdsControl, IdsNetlist  # Add IdsControl and IdsNetlist to the import

# Constants
DESIGN_PREFIX = "mosIds"
DEFAULT_NFINGERS = 1
DEFAULT_MAX_DIFF = 3
DEFAULT_MULTIPLIER = 1
WIDTH_PRECISION = 1  # Decimal places for width in output
LENGTH_PRECISION = 1  # Decimal places for length in output
LOG_FILE = "spfMosIdsCheck.log"
POSTPROCESS_LOG_FILE = "postprocess.log"  # New log file for reusability mode

# Configure logging
def setup_logging(reuse_mode=False, reuse_dir=None):
    """Set up logging configuration."""
    if reuse_mode and reuse_dir:
        log_file = os.path.join(reuse_dir, "reuse_postprocess.log")  # New log file for reuse mode
    else:
        log_file = POSTPROCESS_LOG_FILE if reuse_mode else LOG_FILE

    # Avoid duplicate handlers if logging is already configured
    if logging.getLogger().hasHandlers():
        logging.getLogger().handlers.clear()

    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] %(message)s",
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler(sys.stdout)
        ]
    )
    return logging.getLogger(__name__)

# Ensure logger is initialized globally
logger = setup_logging()

# Global debug messages list for CMOS modeling team
debug_messages = []

def _parse_args():
    """Parse command-line arguments and YAML configuration file.

    Returns:
        tuple: (args dict, config dict) containing command-line args and YAML config.

    Raises:
        FileNotFoundError: If config file doesn't exist.
        yaml.YAMLError: If config file is malformed.
        ValueError: If projRev is missing when required.
    """
    # Define the script description and usage examples
    desc = (
        "Check pre- and post-layout simulations produce similar MOSFET current in saturation.\n"
        "Creates layout in a temporary design in /proj/rtest, extracts SPF, and simulates Ids.\n"
        "Examples:\n"
        "    spfMosIdsCheck -p y52c/REV              <-- Test Ids on NCH W=1.0 NF=1\n"
        "    spfMosIdsCheck -p y52c/REV -f 10 -k     <-- Test Ids on NCH W=10*Wmin NF=1, keep design\n"
        "    spfMosIdsCheck -p y52c/REV -f 10 -x 10  <-- Test Ids on NCH W=10*Wmin NF=10\n"
        "    spfMosIdsCheck -p y52c/REV -d /path/to/expKit  <-- Test Ids using expKit\n"
        "    spfMosIdsCheck -c config.yaml           <-- Use config file for multiple tests"
    )
    parser = argparse.ArgumentParser(description=desc, formatter_class=argparse.RawTextHelpFormatter)
    parser.add_argument('-p', '--projRev', help='Project/revision, e.g., z21c/REV (optional if using -c with projRev in config)')
    parser.add_argument('-d', '--dtkDir', help='Path to DTK directory')
    parser.add_argument('-t', '--maxDiffPercent', type=float, help=f'Max Ids difference percent (default={DEFAULT_MAX_DIFF})')
    parser.add_argument('-s', '--siIniFile', help='Path to si.ini file')
    parser.add_argument('-m', '--model', help='N-type model (default=NCH)')
    parser.add_argument('-f', '--minGateWMultiplier', help='Gate width multiplier (default=1.0)')
    parser.add_argument('-n', '--processName', help='Process name (e.g., 150series)')
    parser.add_argument('--showProcessInfo', help='Print process info and exit')
    parser.add_argument('-x', '--numberOfFingers', type=int, help=f'Number of fingers (default={DEFAULT_NFINGERS})')
    parser.add_argument('-v', '--sweepVoltage', type=float, help=f'Sweep voltage (default={MosfetIdsCheck.DEFAULT_HIGH_VOLTAGE})')
    parser.add_argument('-e', '--lle', action='store_true', help='Enable LLE parameter')
    parser.add_argument('-k', '--keep', action='store_true', help='Keep temporary database')
    parser.add_argument('-c', '--config', help='Path to YAML config file')
    parser.add_argument('--html', action='store_true', help='Generate cetest.html with debug messages')
    parser.add_argument('--reuse', help='Path to existing run directory for reusing simulation results')

    args = vars(parser.parse_args())
    config = {}

    if args['config']:
        if not os.path.isfile(args['config']):
            raise FileNotFoundError(f"Config file '{args['config']}' not found")
        with open(args['config'], 'r') as file:
            try:
                config = yaml.safe_load(file) or {}
            except yaml.YAMLError as e:
                raise yaml.YAMLError(f"Invalid YAML in '{args['config']}': {e}")

        if any(arg for arg_name, arg in args.items() if arg is not None and arg_name not in ['config', 'projRev', 'html']):
            raise ValueError("When using -c, only -p/--projRev and --html are allowed alongside it")

        args['projRev'] = args['projRev'] or config.get('projRev')
        if not args['projRev']:
            raise ValueError("projRev must be provided in config.yaml or via -p/--projRev when using -c")

    if args['reuse']:
        if args['config'] or args['projRev']:
            raise ValueError("When using --reuse, -c and -p/--projRev should not be specified")
        if not os.path.isdir(args['reuse']):
            raise FileNotFoundError(f"Run directory '{args['reuse']}' not found")
        run_log = os.path.join(args['reuse'], 'run.log')
        config_file = os.path.join(args['reuse'], 'config.yaml')
        if not os.path.isfile(run_log):
            raise FileNotFoundError(f"run.log not found in '{args['reuse']}'")
        if not os.path.isfile(config_file):
            raise FileNotFoundError(f"config.yaml not found in '{args['reuse']}'")
        with open(config_file, 'r') as file:
            try:
                config = yaml.safe_load(file) or {}
            except yaml.YAMLError as e:
                raise yaml.YAMLError(f"Invalid YAML in '{config_file}': {e}")
        args['projRev'] = config.get('projRev')

    if not args['config'] and not args['projRev'] and not args['reuse']:
        raise ValueError("-p/--projRev is required when not using -c or --reuse")

    return args, config

def check_env(projRev):
    """Verify environment consistency with project/revision.

    Args:
        projRev (str): Project/revision identifier (e.g., 'y52k/REV').

    Raises:
        SystemExit: If CE_PROJECT environment variable mismatches projRev.
    """
    # Ensure the environment is correctly set up for the specified project/revision
    if os.getenv('CEENV_LOADED'):
        env_proj = os.getenv('CE_PROJECT')
        if env_proj != projRev:
            logger.error(f"Environment proj/rev '{env_proj}' does not match '{projRev}'")
            sys.exit(35)

def generate_debug_message(test_name, result, max_diff=DEFAULT_MAX_DIFF):
    """Generate a debug message summarizing the test result.

    Args:
        test_name (str): Name of the test.
        result (str): Result of the test (e.g., "Passed!" or error details).
        max_diff (float): Maximum allowed difference percentage.

    Returns:
        str: Debug message summarizing the test result.
    """
    if "Passed" in result:
        parts = result.split('Passed! ', 1)
        details = parts[1] if len(parts) > 1 else "No additional details"
        message = (
            f"Test {test_name} passed: Pre- and post-layout Ids match within "
            f"{max_diff}% tolerance. Result: {details}"
        )
    else:
        details = result.split(' ', 1)[1] if ' ' in result else "Unknown failure"
        message = (
            f"Test {test_name} failed: {details}. Check SPF extraction "
            f"(/proj/rtest/.../spfrun/) and simulation logs "
            f"(/proj/rtest/.../simrun/) for CMOS model vs. SPF mismatch."
        )
    debug_messages.append(message)
    return message

def extract_temp_design_paths(log_file):
    """Extract temporary design paths from run.log."""
    pattern = r"Created temporary design at: (\S+)"
    with open(log_file, 'r') as f:
        content = f.read()
    return re.findall(pattern, content)

class RunMosfetIdsCheck:
    """Class to manage MOSFET Ids pre- vs post-layout comparison.

    Attributes:
        originalProjRev (str): Original project/revision identifier (e.g., 'y52k/REV').
        projRev (str): Sanitized project/revision identifier (e.g., 'y52k_REV').
        projDir (str): Project directory path.
        dtk (DTK): DTK instance for simulation.
        cellName (str): Name of the cell for simulation.
        libName (str): Name of the library for simulation.
        numberOfFingers (int): Number of fingers for the MOSFET.
        modelName (str): Model name of the MOSFET (e.g., 'NCH').
        processName (str): Process series (e.g., '150series').
        keep (bool): Whether to retain temporary designs.
        gateWidth (float): Explicit gate width from config, if provided.
        gateLength (float): Explicit gate length from config, if provided.
    """

    DESIGN_PREFIX = "mosIds"
    DEFAULT_NFINGERS = 1
    DEFAULT_MAX_DIFF = 3
    DEFAULT_MULTIPLIER = 1

    def __init__(self, projRev, config=None, **kwargs):
        """Initialize the RunMosfetIdsCheck instance.

        Args:
            projRev (str): Project/revision (e.g., 'y52k/REV').
            config (dict, optional): Configuration from YAML file.
            **kwargs: Additional command-line arguments.

        Raises:
            ValueError: If projRev is invalid or missing required fields.
        """
        # Validate and initialize project/revision
        if not projRev or not isinstance(projRev, str):
            raise ValueError("projRev must be a non-empty string")
        self.originalProjRev = projRev
        self.projRev = projRev.replace('/', '_')
        check_env(self.originalProjRev)
        self.proj = self.originalProjRev.split('/')[0]
        self.projDir = os.path.join(os.environ.get('MS_PUBLIC_PROJ_DIR', '/default'), self.originalProjRev)

        # Initialize DTK directory and related configurations
        self.siIniFile = kwargs.get('siIniFile', config.get('siIniFile') if config else None)
        dtkDir = kwargs.get('dtkDir', config.get('dtkDir') if config else None)
        if dtkDir is None:
            dtkDir = Model.getDTKDirFromIni(self.siIniFile) or os.path.join(self.projDir, "libs", "TECHLIB")
        if not os.path.isdir(dtkDir):
            logger.warning(f"DTK directory '{dtkDir}' not found; may cause issues")
        logger.info(f"Initializing with DTK from: {dtkDir}")
        self._dtk = DTK(dtkDir)

        # Set default and user-provided parameters
        self.commonGateWFactor = float(kwargs.get('minGateWMultiplier', config.get('minGateWMultiplier', self.DEFAULT_MULTIPLIER) if config else self.DEFAULT_MULTIPLIER))
        self.fixedWidthValue = float(kwargs.get('fixedWidthValue', 1.0))
        self.maxSPFToPrelayouDiff = float(kwargs.get('maxDiffPercent', config.get('maxDiffPercent', self.DEFAULT_MAX_DIFF) if config else self.DEFAULT_MAX_DIFF))
        self.numberOfFingers = int(kwargs.get('numberOfFingers', config.get('fingers', self.DEFAULT_NFINGERS) if config else self.DEFAULT_NFINGERS))
        self.modelName = kwargs.get('model', config.get('modelName', 'NCH') if config else 'NCH')
        self.processName = kwargs.get('processName', config.get('processName') if config else None)
        self.lle = kwargs.get('lle', config.get('lle', False) if config else False)
        self.keep = kwargs.get('keep', config.get('keep', False) if config else False)
        self.showProcessInfo = kwargs.get('showProcessInfo', config.get('showProcessInfo', False) if config else False)
        self.simCheckVoltage = float(kwargs.get('sweepVoltage', config.get('sweepVoltage', MosfetIdsCheck.DEFAULT_HIGH_VOLTAGE) if config else MosfetIdsCheck.DEFAULT_HIGH_VOLTAGE))
        self._buildLayoutOnly = False
        self._preLayoutOnly = False
        logger.info(f"Keep temporary designs: {self.keep}")

    def run(self, device_name=None, fingers=None, width=None, length=None):
        """Execute the MOSFET Ids comparison test.

        Args:
            device_name (str, optional): Device identifier (e.g., 'nmos').
            fingers (int, optional): Number of fingers.
            width (float, optional): Gate width in microns.
            length (float, optional): Gate length in microns.

        Returns:
            str: Result string (e.g., "Passed! Vds=...").

        Raises:
            ValueError: If critical parameters are invalid.
        """
        # Validate and set test parameters
        if fingers is not None:
            self.numberOfFingers = int(fingers)
        if width is not None:
            self.gateWidth = float(width)
        if length is not None:
            self.gateLength = float(length)

        if self.numberOfFingers < 1:
            raise ValueError(f"Number of fingers must be positive, got {self.numberOfFingers}")

        w_str = f"{self.gateWidth:.1f}" if hasattr(self, 'gateWidth') and self.gateWidth is not None else f"{self.commonGateWFactor:.1f}"
        l_str = f"{self.gateLength:.1f}" if hasattr(self, 'gateLength') and self.gateLength is not None else "default"
        self.cellName = f"{self.projRev}_{self.numberOfFingers}_{self.modelName}_W{w_str}_L{l_str}"
        self.libName = f"{self.projRev}_{self.numberOfFingers}_{self.modelName}_W{w_str}_L{l_str}"

        width_display = self.gateWidth if hasattr(self, 'gateWidth') and self.gateWidth is not None else self.commonGateWFactor
        length_display = self.gateLength if hasattr(self, 'gateLength') and self.gateLength is not None else 'default'
        logger.info(f"Running test for projRev: {self.projRev}, model: {self.modelName}, fingers: {self.numberOfFingers}, width: {width_display}, length: {length_display}")
        self._createTempDesign()
        logger.info(f"Created temporary design at: {self._tmpDesign.buildPath}")

        self._db = DesignDatabase(self._tmpDesign.cdsLib(), True)
        self._db.openLibs()

        self._createTestLibrary()
        logger.info(f"Created test library: {self.libName}")
        self._mosCheck = MosfetIdsCheck(self._dtk, self._db, self.originalProjRev)
        if self.processName is not None:
            self._mosCheck.processName = self.processName
            logger.info(f"Set processName: {self.processName}")
        self._mosCheck.nf = self.numberOfFingers
        self._mosCheck.libName = self.libName
        self._mosCheck.cellName = self.cellName
        self._mosCheck.customSiIniFile = self.siIniFile
        self._mosCheck.use3D = False
        self._mosCheck.simCheckVoltage = self.simCheckVoltage
        self._mosCheck.lle = self.lle
        self._createLayout()
        logger.info("Layout created successfully")
        if self._buildLayoutOnly:
            logger.info("Finished, only building the layout")
            sys.exit(0)
        if self._preLayoutOnly:
            logger.info("Running pre-layout simulation only")
            return self._results()
        else:
            logger.info(f"{DateTime.cast(DateTime.now()).toStr('%c')} Begin SPF extract...")
            with self._tmpDesign.loadEnv():
                spfRunStatus = self._mosCheck.runSPF(self._tmpDesign.buildPath + "/spfrun", self.tmpProjRev)
            if spfRunStatus == 0:
                logger.info(f"{DateTime.cast(DateTime.now()).toStr('%c')} SPF finished successfully, starting Ids sims...")
                self._runSPFSim()
                self._runPreLayoutSim()
                result = self._results()
                logger.info(f"Test completed: {result}")
                return result
            else:
                logger.error("SPF did not finish successfully")
                os.system(f"cat {self._tmpDesign.buildPath}/spfrun/{self._mosCheck.cellName}/si.log")
                return "ERROR: SPF extraction failed"
        self._cleanUp()

    def _runSPFSim(self):
        """Run post-layout SPF simulation.

        Raises:
            RuntimeError: If simulation fails.
        """
        logger.info(f"Running SPF simulation for {self.cellName}")
        status = self._mosCheck.runSPFSim(os.path.join(self._tmpDesign.buildPath, "simrun", self.cellName))
        if status:
            error_msg = f"ERROR: SPF sim failed with status {status}"
            logger.error(error_msg)
            raise RuntimeError(error_msg)
        logger.info("SPF simulation completed")

    def _runPreLayoutSim(self):
        """Run pre-layout simulation.

        Raises:
            RuntimeError: If simulation fails.
        """
        logger.info(f"Running pre-layout simulation for {self.cellName}")
        status = self._mosCheck.runPreLayoutSim(os.path.join(self._tmpDesign.buildPath, "simrun", self.cellName))
        if status:
            error_msg = f"ERROR: Pre-layout sim failed with status {status}"
            logger.error(error_msg)
            raise RuntimeError(error_msg)
        logger.info("Pre-layout simulation completed")

    def _createTempDesign(self):
        """Create a temporary design directory for simulation.

        Raises:
            OSError: If directory creation fails.
        """
        # Set up a temporary design directory for simulation
        buildPath = os.path.join("/proj", "rtest")
        self._tmpDesign = TmpDesign(self.projDir, buildPath, f"{self.DESIGN_PREFIX}_{self.proj}_")
        try:
            os.chmod(self._tmpDesign.buildPath, 0o2775)
        except OSError as e:
            logger.error(f"Failed to set permissions on {self._tmpDesign.buildPath}: {e}")
            raise
        self.tmpProjRev = self._tmpDesign.projectRev()
        self._tmpDesign.projRev = self.tmpProjRev
        self._tmpDesign.read()
        self._tmpDesign.populate()
        self._tmpDesign.writeEnvSelectFile(self.originalProjRev)
        self._df2Executable()

    def _df2Executable(self):
        """Create a start.df2 executable if keeping designs.

        Raises:
            OSError: If file creation or permission setting fails.
        """
        if self.keep:
            exeFile = "start.df2"
            with open(exeFile, "w") as fh:
                if os.getenv('CEENV_LOADED'):
                    fh.write("#!/bin/bash -i\n")
                    fh.write(f"ceenv {self.tmpProjRev}\n")
                    fh.write("df2\n")
                else:
                    fh.write("#!/bin/bash\n")
                    fh.write(f"df2 -p {self.tmpProjRev}\n")
            try:
                st = os.stat(exeFile)
                os.chmod(exeFile, st.st_mode | stat.S_IEXEC)
                logger.info(f"Created executable start.df2 for temporary design: {self.tmpProjRev}")
            except OSError as e:
                logger.error(f"Failed to create start.df2: {e}")
                raise

    def _createTestLibrary(self):
        """Create the test library for simulation.

        Raises:
            RuntimeError: If library creation fails.
        """
        # Create a library in the temporary design for simulation
        lib_path = os.path.join(self._tmpDesign.buildPath, "libs", self.libName)
        logger.info(f"Creating Library {self.libName} at {lib_path}")
        try:
            self._db.createLib(self.libName, lib_path)
            self._db.makeEditable()
            self._db.writeLibDefToCdsLib(self.libName, lib_path)
            self._db.libWriteAccess(self.libName)
        except Exception as e:
            logger.error(f"Failed to create library {self.libName}: {e}")
            raise RuntimeError(f"Library creation failed: {e}")

    def _createLayout(self):
        """Generate the MOSFET layout.

        Raises:
            ValueError: If layout parameters are invalid.
        """
        # Generate the layout for the MOSFET based on the provided parameters
        process = Process(self._dtk)
        devByType = self._selectModel(process.mosDevicesByType())
        allModels = [m for (k, s) in devByType.items() for m in s]
        if hasattr(self, 'gateWidth') and self.gateWidth is not None:
            self._mosCheck.commonGateW = self.gateWidth
        elif self.fixedWidthValue is None:
            self._mosCheck.commonGateW = process.commonMin('wmin', allModels) / process.ONE_MICRON * self.commonGateWFactor
        else:
            self._mosCheck.commonGateW = self.fixedWidthValue

        if hasattr(self, 'gateLength') and self.gateLength is not None:
            self._mosCheck.commonGateL = self.gateLength
        else:
            self._mosCheck.commonGateL = process.commonMin(Process.L_MIN, allModels) / process.ONE_MICRON

        if self._mosCheck.commonGateW <= 0 or self._mosCheck.commonGateL <= 0:
            raise ValueError(f"Invalid layout dimensions: W={self._mosCheck.commonGateW}, L={self._mosCheck.commonGateL}")

        self._mosCheck.createLayout(devByType)

    def _selectModel(self, devByType):
        """Select the MOSFET model from the device list.

        Args:
            devByType (dict): Dictionary of device types from DTK.

        Returns:
            dict: Filtered device type dictionary.

        Raises:
            ValueError: If modelName is invalid.
        """
        nModel = "NCH" if self.modelName is None else self.modelName
        if self.modelName is None:
            modelConfig = self._dtk.ModelInfo.getConfig()
            if modelConfig.hasParam("DTK_DEFAULT_N_MODEL"):
                nModel = modelConfig.param("DTK_DEFAULT_N_MODEL")
            self.modelName = nModel

        tmpDBT = {Process.N_TYPE: set(), Process.P_TYPE: set()}
        if nModel in devByType[Process.N_TYPE]:
            tmpDBT[Process.N_TYPE].add(nModel)
            logger.info(f"Selected N-type model: {nModel}")
        elif nModel in devByType[Process.P_TYPE]:
            tmpDBT[Process.P_TYPE].add(nModel)
            logger.info(f"Selected P-type model: {nModel}")
        else:
            raise ValueError(f"Invalid model: {nModel} not found in N-type or P-type devices")

        return tmpDBT

    def _results(self):
        """Get simulation results.

        Returns:
            str: Result string or 'Passed!' if no result.
        """
        # Retrieve and return the simulation results
        result = self._mosCheck.printSimResults(self.maxSPFToPrelayouDiff)
        return result if result else "Passed!"


    def _cleanUp(self):
        """Clean up temporary design directories if not keeping them."""
        # Remove temporary design directories unless the 'keep' flag is set
        if hasattr(self, '_db'):
            self._db.libReleaseAccess(self.libName)
        if not self.keep and hasattr(self, '_tmpDesign'):
            logger.info(f"Cleaning up temporary design: {self._tmpDesign.buildPath}")
            shutil.rmtree(self._tmpDesign.buildPath, ignore_errors=True)
        elif self.keep:
            logger.info(f"Keeping temporary design at: {self._tmpDesign.buildPath}")

def main():
    """Main entry point for the script."""
    try:
        # Parse arguments and configuration
        args, config = _parse_args()

        # Initialize logging
        logger = setup_logging(reuse_mode=bool(args['reuse']), reuse_dir=args.get('reuse'))

        logger.info(f"{DateTime.cast(DateTime.now()).toStr('%c')} Start Ids pre-vs-post layout check")
        all_results = []
        generate_html = args.get('html', False) or config.get('generate_html', False)

        if args['reuse']:
            run_dir = args['reuse']
            # ...existing code...
            config_file = os.path.join(run_dir, 'config.yaml')
            with open(config_file, 'r') as file:
                config = yaml.safe_load(file) or {}

            proj_rev = config.get('projRev')
            max_diff = float(config.get('maxDiffPercent', DEFAULT_MAX_DIFF))
            sweep_voltage = float(config.get('sweepVoltage', MosfetIdsCheck.DEFAULT_HIGH_VOLTAGE))
            lle_enabled = config.get('lle', False)
            si_ini_file = config.get('siIniFile')
            process_name = config.get('processName')

            # Initialize DTK and DesignDatabase
            dtk_dir = config.get('dtkDir') or os.path.join(os.environ.get('MS_PUBLIC_PROJ_DIR', '/default'), proj_rev, "libs", "TECHLIB")
            if not os.path.isdir(dtk_dir):
                logger.warning(f"DTK directory '{dtk_dir}' not found; assuming existing sim data is valid")
            dtk = DTK(dtk_dir)
            db = DesignDatabase(None, True)

            # Ensure 'devices' key exists and is not empty
            if 'devices' not in config or not config['devices']:
                logger.error("No 'devices' section found in the configuration or it is empty. Please check config.yaml.")
                sys.exit(1)

            # Extract temporary design paths from spfMosIdsCheck.log
            spf_log = os.path.join(run_dir, 'spfMosIdsCheck.log')
            if not os.path.isfile(spf_log):
                logger.error(f"Log file not found: {spf_log}")
                sys.exit(1)

            temp_design_paths = extract_temp_design_paths(spf_log)
            if not temp_design_paths:
                logger.error("No temporary design paths found in spfMosIdsCheck.log. Ensure the log contains valid paths.")
                sys.exit(1)
            logger.info(f"Extracted temporary design paths: {temp_design_paths}")

            for device in config.get('devices', []):
                model_name = device.get('modelName', 'NCH')
                if 'geometries' not in device or not device['geometries']:
                    logger.warning(f"No geometries found for device '{model_name}'. Skipping this device.")
                    continue

                for geometry in device['geometries']:
                    nf = geometry.get('fingers', DEFAULT_NFINGERS)
                    gate_w = geometry.get('gateWidth')
                    gate_l = geometry.get('gateLength')
                    if gate_w is None or gate_l is None:
                        logger.warning(f"Invalid geometry for device '{model_name}': {geometry}. Skipping this geometry.")
                        continue

                    # Construct cell name and simulation directories
                    w_str = f"{gate_w:.1f}" if gate_w is not None else "default"
                    l_str = f"{gate_l:.1f}" if gate_l is not None else "default"
                    cell_name = f"{proj_rev.replace('/', '_')}_{nf}_{model_name}_W{w_str}_L{l_str}"

                    spf_sim_dir = os.path.join(temp_design_paths[0], 'simrun', cell_name, MosfetIdsCheck.SPF_SIM_DIR)
                    pre_sim_dir = os.path.join(temp_design_paths[0], 'simrun', cell_name, MosfetIdsCheck.SCH_SIM_DIR)

                    # Validate simulation directories
                    if not os.path.isdir(spf_sim_dir):
                        logger.error(f"SPF simulation directory not found: {spf_sim_dir}. Skipping this cell.")
                        continue
                    if not os.path.isdir(pre_sim_dir):
                        logger.error(f"Pre-layout simulation directory not found: {pre_sim_dir}. Skipping this cell.")
                        continue

                    logger.info(f"Processing cell '{cell_name}' with SPF dir '{spf_sim_dir}' and Pre-layout dir '{pre_sim_dir}'.")

                    # Initialize MosfetIdsCheck instance
                    mos_check = MosfetIdsCheck(dtk, db, proj_rev)
                    mos_check.simCheckVoltage = sweep_voltage
                    mos_check.lle = lle_enabled
                    mos_check.modelName = model_name
                    mos_check.nf = nf
                    mos_check.commonGateL = gate_l
                    mos_check.commonGateW = gate_w
                    mos_check.cellName = cell_name
                    mos_check.customSiIniFile = si_ini_file
                    if process_name:
                        mos_check.processName = process_name

                    sim_cond = mos_check._simCondition()
                    spf_netlist = IdsNetlist(nf=nf)
                    spf_netlist.gateWidth = gate_w
                    ids_control_spf = IdsControl(spf_netlist, sim_cond)

                    pre_netlist = IdsNetlist(nf=nf)
                    pre_netlist.gateWidth = gate_w
                    ids_control_pre = IdsControl(pre_netlist, sim_cond)

                    mos_check._fsMosIdsSimSPF = FinesimMosfetIdsSim(spf_sim_dir, ids_control_spf, proj_rev)
                    mos_check._fsMosIdsSimPRE = FinesimMosfetIdsSim(pre_sim_dir, ids_control_pre, proj_rev)

                    try:
                        result = mos_check.printSimResults(max_diff)
                        result = result if result else "Passed!"
                        logger.info(f"Processed {cell_name}: {result}")
                        generate_debug_message(cell_name, result, max_diff)
                        logger.info(debug_messages[-1])
                        all_results.append(f"{cell_name}/run.log: {result}")
                    except (ValueError, ZeroDivisionError) as e:
                        logger.error(f"Error processing cell '{cell_name}': {e}")
                        generate_debug_message(cell_name, f"ERROR: {e}", max_diff)
                        logger.error(debug_messages[-1])
                        all_results.append(f"{cell_name}/run.log: ERROR - {e}")
                    except Exception as e:
                        logger.error(f"Unexpected error processing cell '{cell_name}': {e}")
                        logger.error(f"Traceback: {traceback.format_exc()}")
                        generate_debug_message(cell_name, f"ERROR: Unexpected error - {e}", max_diff)
                        logger.error(debug_messages[-1])
                        all_results.append(f"{cell_name}/run.log: ERROR - Unexpected error")

            # Generate reports
            logger.info("\nPost-processing results...")
            errors = [r for r in all_results if "ERROR" in r or "Failed" in r]
            if errors:
                logger.error("Errors encountered:")
                for error in errors:
                    logger.error(error)
            else:
                logger.info("No errors encountered")

            with open(os.path.join(run_dir, "reuse_results.out"), "w") as post_out:
                post_out.write("results:\n")
                for r in all_results:
                    status = r.split(":")[1].strip().split()[0]
                    post_out.write(f"{r.split(':')[0]}:{status}\n")
                post_out.write("\n")

            with open(os.path.join(run_dir, "reuse_summary.txt"), "w") as post_summary:
                post_summary.write("summary:\n")
                for r in all_results:
                    post_summary.write(f"{r}\n")

            if generate_html:
                with open(os.path.join(run_dir, "reuse_cetest.html"), "w") as html_file:
                    html_file.write("<html><body><h2>Postprocess Test Results</h2><table border='1'>\n")
                    html_file.write("<tr><th>Test Name</th><th>Status</th><th>Details</th></tr>\n")
                    for message in debug_messages:
                        test_name = message.split("Test ")[1].split(" passed")[0] if "passed" in message else message.split("Test ")[1].split(" failed")[0]
                        status = "Passed" if "passed" in message.lower() else "Failed"
                        details = message.split(": ", 1)[1] if ": " in message else "No details"
                        html_file.write(f"<tr><td>{test_name}</td><td>{status}</td><td>{details}</td></tr>\n")
                    html_file.write("</table></body></html>")

        else:
            # Regular flow (unchanged)
            if config and 'devices' in config:
                for device in config.get('devices', []):
                    for geometry in device.get('geometries', []):
                        runMosCheck = RunMosfetIdsCheck(args['projRev'], config=config)
                        runMosCheck.modelName = device.get('modelName', 'NCH')
                        runMosCheck.numberOfFingers = geometry.get('fingers', runMosCheck.DEFAULT_NFINGERS)
                        runMosCheck.gateWidth = geometry.get('gateWidth')
                        runMosCheck.gateLength = geometry.get('gateLength')
                        runMosCheck.fixedWidthValue = None
                        width_display = runMosCheck.gateWidth if runMosCheck.gateWidth is not None else runMosCheck.commonGateWFactor
                        length_display = runMosCheck.gateLength if runMosCheck.gateLength is not None else 'default'
                        logger.info(f"\nStarting test for {device.get('name', 'unknown')} with fingers={runMosCheck.numberOfFingers}, width={width_display}, length={length_display}")
                        result = runMosCheck.run(device.get('name', 'unknown'), runMosCheck.numberOfFingers, runMosCheck.gateWidth, runMosCheck.gateLength)
                        test_name = f"{runMosCheck.projRev}_{runMosCheck.numberOfFingers}_{runMosCheck.modelName}_W{width_display}_L{length_display}"
                        generate_debug_message(test_name, result, runMosCheck.maxSPFToPrelayouDiff)
                        logger.info(debug_messages[-1])
                        all_results.append(f"{test_name}/run.log: {result}")
                        if not runMosCheck.keep:
                            runMosCheck._cleanUp()

                logger.info("\nPost-processing results...")
                errors = [r for r in all_results if "ERROR" in r]
                if errors:
                    logger.error("Errors encountered:")
                    for error in errors:
                        logger.error(error)
                else:
                    logger.info("No errors encountered")

                with open("run.out", "w") as run_out:
                    run_out.write("results:\n")
                    for r in all_results:
                        status = r.split(":")[1].strip().split()[0]
                        run_out.write(f"{r.split(':')[0]}:{status}\n")
                    run_out.write("\n")  # Add trailing newline to match golden.out

                with open("summary.txt", "w") as summary_txt:
                    summary_txt.write("summary:\n")
                    for r in all_results:
                        summary_txt.write(f"{r}\n")

                if all_results:
                    logger.info("\nSummary of results:")
                    for r in all_results:
                        logger.info(r)
                else:
                    logger.info("\nNo results to summarize")

                if generate_html:
                    with open("cetest.html", "w") as html_file:
                        html_file.write("<html><body><h2>Test Results</h2><table border='1'>\n")
                        html_file.write("<tr><th>Test Name</th><th>Status</th><th>Details</th></tr>\n")
                        for message in debug_messages:
                            test_name = message.split("Test ")[1].split(" passed")[0] if "passed" in message else message.split("Test ")[1].split(" failed")[0]
                            status = "Passed" if "passed" in message.lower() else "Failed"  # Fixed status logic
                            details = message.split(": ", 1)[1] if ": " in message else "No details"
                            html_file.write(f"<tr><td>{test_name}</td><td>{status}</td><td>{details}</td></tr>\n")
                        html_file.write("</table></body></html>")

            else:
                runMosCheck = RunMosfetIdsCheck(args['projRev'],
                                                siIniFile=args.get('siIniFile'),
                                                dtkDir=args.get('dtkDir'))
                runMosCheck.keep = args.get('keep', False)
                runMosCheck.lle = args.get('lle', False)
                if args.get('maxDiffPercent') is not None:
                    runMosCheck.maxSPFToPrelayouDiff = float(args['maxDiffPercent'])
                if args.get('minGateWMultiplier') is not None:
                    runMosCheck.commonGateWFactor = float(args['minGateWMultiplier'])
                    runMosCheck.fixedWidthValue = None
                if args.get('model') is not None:
                    runMosCheck.modelName = args['model']
                if args.get('processName') is not None:
                    runMosCheck.processName = args.get('processName')
                if args.get('numberOfFingers') is not None:
                    runMosCheck.numberOfFingers = int(args.get('numberOfFingers'))
                if args.get('sweepVoltage') is not None:
                    runMosCheck.simCheckVoltage = float(args.get('sweepVoltage'))

                logger.info("\nStarting single test with command-line arguments")
                result = runMosCheck.run()
                test_name = f"{runMosCheck.projRev}_{runMosCheck.numberOfFingers}_{runMosCheck.modelName}_W{runMosCheck.commonGateWFactor}_L{runMosCheck.gateLength or 'default'}"
                generate_debug_message(test_name, result, runMosCheck.maxSPFToPrelayouDiff)
                logger.info(f"Test result: {result}")
                logger.info(debug_messages[-1])
                all_results.append(f"{test_name}/run.log: {result}")

                with open("run.out", "w") as run_out:
                    run_out.write("results:\n")
                    status = result.split()[0]
                    run_out.write(f"{test_name}/run.log:{status}\n")
                    run_out.write("\n")  # Add trailing newline to match golden.out

        logger.info(f"{DateTime.cast(DateTime.now()).toStr('%c')} Done")
    except Exception as e:
        logger.error(f"Execution failed: {e}")
        sys.exit(1)

if __name__ == "__main__":
    main()

In [ ]:
############################################################################### filename: MosfetIdsCheck.py
# Description: For testing Gate-to-Drain Cap assumption in CMOS models
# Programmer : Brady Cannon
###############################################################################

from __future__ import print_function

import os
import re
from shutil import copyfile

from Model.Sim.FinesimMosfetIdsSim import FinesimMosfetIdsSim, IdsControl, IdsNetlist, IdsSimCondition
from .MosfetCheck import MosfetCheck
from Model.Device.MosfetDimensions import MosfetDimensions
from Model.Device.MosfetLayoutCellView import MosfetLayoutCellView
import logging
logger = logging.getLogger(__name__)


class MosfetIdsCheck(MosfetCheck):
    '''
    The saturation current for transistors is expected to be very similar in pre and post-layout simulations.
    We know the post-layout simulation depends on the routing to the device, but we first want to ensure the
    sim produces the same result when routing is minimal/ideal.
    This is intended to help prevent issues like we saw on y52c in CADHELP-462.
    To check this, we run a .dc simulation on both SPF and Pre-layout netlists and expect the results to be within a small % of
    each other.
    '''
    CAP_NET_RE = re.compile("^[NP]")
    PWELL_NET = "VSS"
    SPF_SIM_DIR = "finesim_ids.spf"
    SCH_SIM_DIR = "finesim_ids.pre"
    VOLT_CRITERIA_POINTS = [0.0, 1.0]  # 0v and 1.0v
    DEFAULT_HIGH_VOLTAGE = 1.0

    def __init__(self, dtk, designDatabase, projRev):
        super().__init__(dtk, designDatabase, projRev)
        self.libName = "test_spf"
        self.cellName = "mos_ids"
        self._defaultProcessName()
        self.nf = 1
        self.simCheckVoltage = self.DEFAULT_HIGH_VOLTAGE
        self.lle = False  # Initialize the lle attribute

    def _defaultProcessName(self):
        """
        processName must match a pre-defined value in Model/Device/MosfetDimensions.py
        """
        processNumber = self._dtk.Env.getProcessSeries()
        self.processName = "{}series".format(processNumber)

    def runSPFSim(self, topDir="."):
        """
        Only simulating one NMOS device
        """
        testStruct = self.testStructures[0]
        netlist = IdsNetlist(netlistFile="{}/cell.spf".format(self._spfRunDir), nf=self.nf)
        netlist.gateWidth = testStruct.gateW
        netlist.drainNet = testStruct.drainPinText
        netlist.gateNet = testStruct.gateNet
        netlist.sourceNet = testStruct.sourcePinText
        netlist.bulkNet = self.PWELL_NET
        simCond = self._simCondition()
        idsControl = IdsControl(netlist, simCond)
        self._spfSimDir = topDir + "/" + self.SPF_SIM_DIR
        (self._fsMosIdsSimSPF, status) = self._runSim(self._spfSimDir, idsControl)
        return status

    def runPreLayoutSim(self, topDir="."):
        """
        Only simulating one NMOS device
        """
        testStruct = self.testStructures[0]
        netlist = IdsNetlist(deviceName=testStruct.model, nf=self.nf)
        netlist.gateWidth = testStruct.gateW
        netlist.gateLength = testStruct.gateL
        netlist.drainNet = testStruct.drainPinText
        netlist.gateNet = testStruct.gateNet
        netlist.sourceNet = testStruct.sourcePinText
        netlist.bulkNet = self.PWELL_NET
        simCond = self._simCondition()
        idsControl = IdsControl(netlist, simCond)
        self._preLayoutSimDir = topDir + "/" + self.SCH_SIM_DIR
        (self._fsMosIdsSimPRE, status) = self._runSim(self._preLayoutSimDir, idsControl)
        return status

    def _simCondition(self):
        simCond = IdsSimCondition(self._dtk.Env.getPath())
        simCond.drainVoltages = [self.simCheckVoltage]
        simCond.vg = self.simCheckVoltage
        simCond.lle = self.lle
        return simCond

    def spfSimResult(self):
        result = self._fsMosIdsSimSPF.result()
        if not result:
            logger.error("SPF simulation result is empty or missing.")
            raise ValueError("SPF simulation result is empty or missing.")
        logger.info(f"SPF simulation result: {result}")
        return result

    def preLayoutSimResult(self):
        result = self._fsMosIdsSimPRE.result()
        if not result:
            logger.error("Pre-layout simulation result is empty or missing.")
            raise ValueError("Pre-layout simulation result is empty or missing.")
        logger.info(f"Pre-layout simulation result: {result}")
        return result

    def printSimResults(self, maxPercentDiffAllowed):
        (vsat, preIds, spfIds, diff, percent, gateW) = self._simData()
        message = " Vds=Vgs={} W={} L={} NF={} Sch_Ids={:.2f} SPF_Ids={:.2f} Diff: {:.2f}% {:.2f} uA/um".format(
            vsat, gateW * self.nf, self.commonGateL, self.nf, preIds, spfIds, percent, diff)
        if abs(percent) > maxPercentDiffAllowed:
            print("Failed! {}".format(message))
        else:
            print("Passed! {}".format(message))

    def _runSim(self, simDir, idsControl):
        """
        Only simulating one NMOS device
        """
        if not os.path.exists(simDir):
            os.makedirs(simDir)
        self._siIni(simDir)
        fsMosIdsSim = FinesimMosfetIdsSim(simDir, idsControl, self.projRev)
        fsMosIdsSim.setup()
        return (fsMosIdsSim, fsMosIdsSim.sim())

    def _siIni(self, runDir):
        """
        Create si.ini, using custom si.ini if requested
        """
        siIniFile = runDir + os.sep + "si.ini"
        if self.customSiIniFile is not None and os.path.isfile(self.customSiIniFile):
            print("Using custom si.ini file:", self.customSiIniFile)
            copyfile(self.customSiIniFile, siIniFile)
        else:
            open(siIniFile, 'a').close()
        with open(siIniFile, 'a') as fh:
            print("PM.DTK_DIR = \"{}\"".format(self._dtk.Env.getPath()), file=fh)
        return siIniFile

    def _disableVirtualRouteFill(self, siIniFile):
        """
        Disable Virtual Route Fill, the old fashioned way
        At this time both of these variables are used.  Requested one to be removed...
        """
        with open(siIniFile, "a") as fh:
            print("_MS_SI_LVS_VIRTUAL_ROUTE_FILL = nil", file=fh)
            print("_MS_SI_SPF_VIRTUAL_ROUTE_FILL = nil", file=fh)

    def _simData(self):
        """
        Collect the SPF and PreLayout sim data, plus diff and %diff at each sweep point
        :return: (vsat, preIds, spfIds, diff, percent, gateW)
        :rtype: tuple
        """
        spf_result = self.spfSimResult()
        pre_result = self.preLayoutSimResult()

        if not spf_result:
            logger.error("SPF simulation results are missing or invalid. Ensure SPF simulation completed successfully.")
            raise ValueError("SPF simulation results are missing or invalid.")
        if not pre_result:
            logger.error("PreLayout simulation results are missing or invalid. Ensure PreLayout simulation completed successfully.")
            raise ValueError("PreLayout simulation results are missing or invalid.")

        try:
            if not isinstance(spf_result, tuple) or len(spf_result) < 3:
                raise ValueError(f"SPF simulation result is not in the expected format: {spf_result}")
            if not isinstance(pre_result, tuple) or len(pre_result) < 3:
                raise ValueError(f"Pre-layout simulation result is not in the expected format: {pre_result}")

            spfIds, vsat, gateW = spf_result
            preIds, vsat, gateW = pre_result
        except ValueError as e:
            logger.error(f"Error unpacking simulation results: {e}")
            raise ValueError("Simulation results are not in the expected format.")

        if preIds == 0:
            logger.error("PreLayout Ids is zero. Cannot calculate percentage difference.")
            raise ZeroDivisionError("PreLayout Ids is zero.")

        diff = spfIds - preIds
        percent = 100.0 * diff / preIds
        logger.info(f"Simulation data: vsat={vsat}, preIds={preIds}, spfIds={spfIds}, diff={diff}, percent={percent}, gateW={gateW}")
        return (vsat, preIds, spfIds, diff, percent, gateW)

    def _isIgnoreNet(self, ignoreNets, netName):
        for reSearch in ignoreNets:
            if reSearch.search(netName):
                return True
        return False

    def _placeInstances(self, mosLayoutCellView, devicesByType):
        mosfetDimensions = MosfetDimensions.byProcess(self.processName)
        print("  gate W (per finger)={}".format(self.commonGateW))
        print("  NF={}".format(self.nf))
        print("  gate L={}".format(self.commonGateL))
        print(mosfetDimensions.toStr(), flush=True)
        for type in devicesByType:
            for model in devicesByType[type]:
                testStruct = MosfetLayoutCellView.createTestStruct(model, self.commonGateW, self.commonGateL, mosfetDimensions, self.nf)
                testStruct.txtHeight = self.textHeight
                testStruct.addSDPins()
                self.testStructures.append(testStruct)
                mosLayoutCellView.drawTestStruct(testStruct)
            mosLayoutCellView.wrapNext()


In [ ]:
############################################################################### filename: FinesimMosfetIdsSim.py
# Description: Run Finesim to simulate transistor saturation current
# Programmer : Brady Cannon
###############################################################################

from __future__ import print_function

import os
import re
from string import Template

from Spice.FinesimDCOutReader import FinesimDCOutReader
import Utils.JobSubmit as JobSubmit

import logging
logger = logging.getLogger(__name__)

class FinesimMosfetIdsSim(object):
    """
    Run Finesim to simulate transistor saturation current
    This is the 'getter done' code that prob needsto be refactored to be re-usable later...
    """
    ONE_MICRON = 1e-6
    SIMULATOR = "finesim"
    W_MIN = "wmin"
    N_TYPE = "NMOS"
    P_TYPE = "PMOS"
    ACCESS_DEV_PTRN = re.compile(".*(Access|Memory).*")
    LOG_FILE = "sim_ids.log"

    def __init__(self, runDir, control, projRev=""):
        self.runDir = runDir
        self.control = control
        self.projRev = projRev

    def setup(self):
        """
        Build the run dir and sim inputs
        :return: True if the run dir was successfully built
        :rtype:
        """
        if not os.path.exists(self.runDir):
            os.makedirs(self.runDir)
        self.control.write(self.runDir)
        return os.path.isdir(self.runDir)

    def sim(self):
        """
        Run the simulation
        :return: status
        :rtype: int
        """
        cwd = os.getcwd()
        if self.runDir:
            os.chdir(self.runDir)
        if os.getenv('CEENV_LOADED'):
            cmd = "{}/sw/bin/finesim {}".format(
                os.getenv('CDE_HOME'), self.control.fileName)
        else:
            cmd = "finesim {}".format(self.control.fileName)
        with open(self.runDir + "/" + self.LOG_FILE, 'w') as logFH:
            print("Running", cmd, file=logFH)
            args = {
                'queue': 'sfinesim',
                'log': 'bsub.log',
                'overwrite_log': True,
                'interactive': True,
                'proj': self.projRev,
                'tool': 'finesim',
                'type': 'spice',
                'job_name': 'mos ids',
                'cmd': cmd
            }
            jsub = JobSubmit.JobSubmit()
            jsub.build(args)
            jsub.submit()
            if jsub.errors:
                for e in jsub.errors:
                    print(e)
            status = jsub.status
        if self.runDir:
            os.chdir(cwd)
        return status

    def result(self):
        """
        Parse the output file and return the simulation result normalized to uA/um.
        :return: (Ids uA/um, VG=VD voltage)
        :rtype: tuple
        """
        fsOutFile = self.runDir + "/finesim.pd0"
        if not os.path.isfile(fsOutFile):
            logger.error(f"Finesim output file not found: {fsOutFile}")
            return None

        try:
            # Log the content of the output file for debugging
            with open(fsOutFile, 'r') as file:
                content = file.read()
                logger.debug(f"Content of Finesim output file '{fsOutFile}':\n{content}")

            fsReader = FinesimDCOutReader([fsOutFile])
            fsReader.parse()
            vsat = self.control.simConditions.vg
            dataTarget = {"vd": vsat, "vg": vsat}
            results = list(fsReader.match(fsOutFile, dataTarget))

            if not results:
                logger.error(f"No results found in Finesim output file: {fsOutFile}")
                logger.debug(f"Expected data target: {dataTarget}")
                return None

            if not isinstance(results[0], dict):
                logger.error(f"Unexpected result format in Finesim output file: {fsOutFile}")
                logger.debug(f"Results content: {results}")
                return None

            ids_value = results[0].get(self.control.IDS_PARAM)
            if ids_value is None:
                logger.error(f"'{self.control.IDS_PARAM}' not found in results: {results[0]}")
                logger.debug(f"Results content: {results}")
                return None

            try:
                ids_value = float(ids_value)
            except ValueError:
                logger.error(f"Invalid Ids value in Finesim output file: {results[0]}")
                return None

            if ids_value == 0:
                logger.warning(f"Ids value is zero in Finesim output file: {fsOutFile}")
            normalized_ids = 1e6 * ids_value / (self.control.netlist.gateWidth * self.control.netlist.nf)
            logger.info(f"Parsed Finesim result: Ids={normalized_ids} uA/um, Vsat={vsat}, GateWidth={self.control.netlist.gateWidth}")
            return normalized_ids, vsat, self.control.netlist.gateWidth
        except Exception as e:
            logger.error(f"Error parsing Finesim output file '{fsOutFile}': {e}")
            return None


class IdsControl(object):
    """
    Represents a control file for simulating Mosfet current in saturation
    IdsControl has-a:
        IdsNetlist
        IdsSimCondition
    """
    SWEEP_NAME = "idvd"
    SWEEP_NETS = ("vg", "vd")
    IDS_PARAM = "mos_ids"
    CONTROL_TEMPLATE = """* SPICE input deck generated by Model.Sim.IdsControl
* Measure Ids in saturation

.lib '$incFile' tt
.lib '$parasiticCnr' TT_rc
***************************************************************************
* .TEMP section
.temp $temperature

.option finesim_mode=spicehd
.option numdgt=7
.option scale=1e-06
.option scalm=1
.option gmin=1e-16
.option gmindc=1e-16
.option CO=132
.option INGOLD=2

.param vd=0.0
.param vg=0.0
.param vs=0.0
.param vb=0.0

.param $lle

v_d $drain 0 dc vd
v_g $gate 0 dc vg
v_s $source 0 dc vs
v_bulk $bulk 0 dc vb

$netlist

** dc sim
.dc data=idvd
.DATA idvd
vd vg vs vb
$vd_sweep_list
.ENDDATA
** For Ids we are interested in the source terminal: i(v_s)
.print dc vd=par('vd') vg=par('vg') vs=par('vs') mos_ids=par(i('v_s'))

.END

"""

    def __init__(self, netlist=None, simConditions=None):
        self.fileName = "control"
        self.netlist = netlist
        self.simConditions = simConditions

    def write(self, directory):
        """
        Write the control file to directory/self.fileName
        :param directory: The simulation directory
        :type directory: str
        :return:
        :rtype:
        """
        template = Template(self.CONTROL_TEMPLATE)
        data = {
            'incFile': self.simConditions.incFile(),
            'parasiticCnr': self.simConditions.parasiticFile(),
            'temperature': self.simConditions.temperature,
            'lle': self.simConditions.lleflag(),
            'width': self.netlist.gateWidth,
            'drain': self.netlist.drainNet,
            'gate': self.netlist.gateNet,
            'source': self.netlist.sourceNet,
            'bulk': self.netlist.bulkNet,
            'netlist': self.netlist.toString(),
            'vd_sweep_list': self.simConditions.drainVoltageSweepList()
        }
        with open(directory + "/" + self.fileName, 'w') as fh:
            fh.write(template.substitute(data))


class IdsSimCondition(object):
    """
    Represents the sim conditions:
        frequency
        ac_voltage
        gate voltage sweep
        drain/source/bulk voltages
        temperature
        DTK model files
    """

    def __init__(self, dtkDir):
        self.temperature = 30
        self._dtkDir = dtkDir
        self.drainVoltages = [x / 10.0 for x in range(-14, 15, 1)]
        self.vb = 0
        self.vg = 0
        self.vs = 0

    def __str__(self):
        stringTmp = """DTK: $dtk
Temp: $temp
"""
        template = Template(stringTmp)
        return template.substitute({'dtk': self._dtkDir, 'temp': self.temperature})

    def incFile(self):
        return self._dtkDir + "/spice.models/incFile"

    def parasiticFile(self):
        return self._dtkDir + "/spice.models/parasitic.cnr"

    def lleflag(self):
        print("lleflag is {}".format(self.lle))
        if self.lle:
            return "WPE_FLAG=1 STI_FLAG=1"
        else:
            return "WPE_FLAG=0 STI_FLAG=0"

    def drainVoltageSweepList(self):
        return "\n".join(["{} {} {} {}".format(vd, self.vg, self.vs, self.vb) for vd in self.drainVoltages])


class IdsNetlist(object):
    """
    Represents the device under test.  Either SPF netlist or pre-layout instance
    """

    def __init__(self, netlistFile=None, deviceName=None, nf=1):
        """
        Either include a netlistFile, or create an instance with deviceName
        """
        self.netlistFile = netlistFile
        self.gateWidth = None
        self.drainNet = None
        self.gateNet = None
        self.sourceNet = None
        self.bulkNet = None

        self.deviceName = deviceName
        self.gateLength = None
        self.nf = nf
        self.geomod = 0
        if self.nf > 1:
            self.geomod = 3

    def toString(self):
        if self.netlistFile:
            return ".inc '{}'\n".format(self.netlistFile)
        elif self.deviceName:
            return "XM0 {} {} {} {} {} w={} l={} ctg_sch=1 nf={} geomod={}\n".format(self.drainNet, self.gateNet, self.sourceNet,
                                                                     self.bulkNet, self.deviceName, self.gateWidth * self.nf,
                                                                     self.gateLength, self.nf, self.geomod)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# prompt: summarize all 3 cells in this notebook

The provided code defines a Python class `FinesimMosfetIdsSim` and related classes (`IdsControl`, `IdsSimCondition`, `IdsNetlist`) for simulating MOSFET saturation current using Finesim.  It includes methods for setting up the simulation environment, running the simulation, and parsing the results.  The code utilizes a control file template and handles various simulation parameters.  Key aspects include reading and writing files, parsing simulation output, and managing simulation conditions and netlists.
